# Project 1
### Epidemiological Study: US Vaccination Campaign (November 2020 - March 2021)
---
### Project Description/Outline
Determine the effectiveness of the US vaccination campaign in attending the population that is most affected by covid-19 disease. This is achieved by revising the total number of doses applied, vaccination coverage, population demographics such as gender, race, socioeconomic status, and education, versus epidemiologic variables: incidence, prevalence, hospitalization, UCI, death numbers.

In [1]:
## Dependencies
import pandas as pd



### Covid-19 Vaccination Data

In [ ]:
## Import data from CSV
hesitancy_df = pd.read_csv("data/Vaccine_Hesitancy_Covid19.csv")
hesitancy_df.head()

In [ ]:
hesitancy_df.iloc[:,10:16].sum(axis=1).mean()*100

In [ ]:
columns = [
    "FIPS Code",
    "Social Vulnerability Index (SVI)",
    "SVI Category",
    "Percent adults fully vaccinated against COVID-19",
    "Percent Hispanic",
    "Percent non-Hispanic American Indian/Alaska Native",
    "Percent non-Hispanic Asian",
    "Percent non-Hispanic Black",
    "Percent non-Hispanic Native Hawaiian/Pacific Islander",
    "Percent non-Hispanic White"
]

vaccination_df = hesitancy_df[columns]
vaccination_df.head()

### Pendiente
- (Mariana) Calcular y agregar Other

In [ ]:
len(vaccination_df)

### US Census Reference (2019)

In [ ]:
census_df = pd.read_csv("data/US_Census2019.csv")
census_df.head()

In [ ]:
census_df.iloc[1:5,15:23].head()

In [ ]:
county_pop_df = census_df[["STATE", "COUNTY","STNAME","CTYNAME", "POPESTIMATE2019"]]
# str(census_df['STATE']) + str(census_df['COUNTY'])

In [ ]:
county_pop_df.head()

### Pendiente
- (María) Separar County=0 en df aparte
- (María) Eliminar County=0 del actual
- (María) Armar y agregar County_FIPS

In [ ]:
county_pop_df['POPESTIMATE2019'].sum()

### Covid-19 Case Surveillance


In [ ]:
cases = pd.read_csv("data/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv", chunksize=1000)

In [ ]:
cases1 = pd.DataFrame(cases)

In [ ]:
import pandas as pd
from sodapy import Socrata


# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("n8mc-b4w4", limit=1000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [ ]:
results_df.tail()

In [ ]:
len(results_df['case_month'].unique())

In [ ]:
results_df.sort_values(['case_month'], ascending=True)

In [ ]:
results_df['sex'].loc[results_df['ethnicity'] != "Hispanic/Latino"].value_counts()

In [ ]:
results_df['age_group'].value_counts()

In [ ]:
columns = [
    "county_fips_code",
    "case_month",
    "current_status",
    "sex",
    "age_group",
    "race",
    "ethnicity",
    "hosp_yn",
    "icu_yn",
    "death_yn"
]

cases_df = results_df[columns]
cases_df.tail()

### Pendientes

- Definir fecha de corte (28/4/2021 = vaccination)
- Eliminar "probable case"

<br>

- (Ric) Cambiar "case_month" a timestamp
- (Ana) ***Nomenclatura AGEGRP de US Census a age_group***
- (Ana) ***Nomenclatura race/ethnicity combined***
- (Ric) Contar número de líneas del CSV Original (Case Surveillance: 24,441,351)
- (Ric) Bajar TODA la información
- (Mariana) ***Revisar Socrata***

In [ ]:
https://data.cdc.gov/resource/n8mc-b4w4.json?case_month=2020-12&current_status=Laboratory-confirmed case

In [2]:
import pandas as pd
import sqlalchemy
from sodapy import Socrata

socrata_domain = 'data.cdc.gov'
socrata_dataset_identifier = "n8mc-b4w4"

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata(socrata_domain, None)

#Get metadata
metadata = client.get_metadata(socrata_dataset_identifier)
[x['name'] for x in metadata['columns']]

['case_month',
 'res_state',
 'state_fips_code',
 'res_county',
 'county_fips_code',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'case_positive_specimen_interval',
 'case_onset_interval',
 'process',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn',
 'underlying_conditions_yn']

In [20]:
results = client.get(socrata_dataset_identifier, limit =2000,
                      #where = "current_status"=="Laboratory-confirmed case",
                     select="county_fips_code,case_month,current_status,sex,age_group,race,ethnicity,hosp_yn,icu_yn,death_yn"
                    )
                    
tryout_df = pd.DataFrame.from_records(results)

In [21]:
tryout_df.head()

,county_fips_code,case_month,current_status,sex,age_group,race,ethnicity,hosp_yn,icu_yn,death_yn
0,NA,2020-08,Laboratory-confirmed case,NA,NA,NA,NA,Missing,Missing,Missing
1,NA,2020-08,Probable Case,NA,NA,NA,NA,Missing,Missing,Missing
2,NA,2020-08,Laboratory-confirmed case,NA,NA,NA,NA,Missing,Missing,Missing
3,NA,2020-08,Laboratory-confirmed case,NA,NA,NA,NA,Missing,Missing,Missing
4,NA,2020-08,Laboratory-confirmed case,NA,NA,NA,NA,Missing,Missing,Missing


In [17]:
tryout_df['current_status'].value_counts()

Laboratory-confirmed case    1716
Probable Case                 284
Name: current_status, dtype: int64